# Import

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from ase.io import read
from ase.visualize.plot import plot_atoms
from mpl_toolkits.mplot3d import Axes3D
import random
import numpy as np
import os
import pandas as pd
import numpy as np
from ase.io import read
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Path setup
data_dir = "./"  # <-- Change this to your actual data path
atoms_train_dir = os.path.join(data_dir, "atoms", "train")
energies_csv = os.path.join(data_dir, "energies", "train.csv")

In [3]:
import numpy as np
import torch
import time
import os

In [4]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

In [5]:
from kymatio.torch import HarmonicScattering3D

In [6]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

In [7]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

In [8]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

# Energies dataframe

In [9]:
# Read energies CSV
energies_df = pd.read_csv(energies_csv)
print("Loaded energy data:\n", energies_df.head().to_string(index=False))

Loaded energy data:
  id     energy
  1 -90.107880
  2 -69.927647
  3 -69.979214
  4 -73.402302
  5 -78.936724


# Positions and charges 

In [10]:
# === Extraction des features depuis les fichiers .xyz ===
def extract_features(xyz_path):
    atoms = read(xyz_path)
    positions = atoms.get_positions()
    charges = atoms.get_atomic_numbers()

    # masses = atoms.get_masses()
    # com = atoms.get_center_of_mass()
    # max_dist = np.linalg.norm(atoms.positions - com, axis=1).max()
    
    return {
        "positions":positions,
        "charges": charges
        # "masse_totale": masses.sum(),
        # "dist_centre_max": max_dist
    }

In [11]:
import numpy as np
import os
from ase.io import read

def extract_features(xyz_path):
    atoms = read(xyz_path)
    positions = atoms.get_positions()
    charges = atoms.get_atomic_numbers()
    return positions, charges

def load_all_xyz(folder_path, max_atoms=23):
    positions_list = []
    charges_list = []
    
    xyz_files = sorted([
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith('.xyz')
    ])

    for xyz_path in xyz_files:
        pos, chg = extract_features(xyz_path)
        
        pos_padded = np.zeros((max_atoms, 3))
        chg_padded = np.zeros((max_atoms,))
        
        n_atoms = pos.shape[0]
        pos_padded[:n_atoms, :] = pos
        chg_padded[:n_atoms] = chg

        positions_list.append(pos_padded)
        charges_list.append(chg_padded)

    positions_array = np.stack(positions_list)  # shape (N, max_atoms, 3)
    charges_array = np.stack(charges_list)      # shape (N, max_atoms)
    
    return positions_array, charges_array


In [12]:
pos, full_charges = load_all_xyz("./atoms/train/", max_atoms=23)
n_molecules = pos.shape[0]

# Data preparation

In [13]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

In [14]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

## Scattering Transform



In [15]:
M, N, O = 32, 32, 32
# M, N, O = 64, 64, 64
# M, N, O = 192, 128, 96


grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [16]:
J = 1
L = 2
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

In [17]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)

HarmonicScattering3D()

In [18]:
device

device(type='cuda')

The maps computed for each molecule are quite large, so the computation has
to be done by batches. Here we select a small batch size to ensure that we
have enough memory when running on the GPU. Dividing the number of molecules
by the batch size then gives us the number of batches.



In [19]:
batch_size = 64
n_batches = int(np.ceil(n_molecules / batch_size))
print(n_batches)

103


We are now ready to compute the scattering transforms. In the following
loop, each batch of molecules is transformed into three maps using Gaussians
centered at the atomic positions, one for the nuclear charges, one for the
valence charges, and one with their difference (called the “core” charges).
For each map, we compute its scattering transform up to order two and store
the results.



In [20]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 6591 molecules from the QM7 database on GPU
sigma: 2.0, L: 2, J: 1, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:03:26]
Iteration 3 ETA: [00:02:47]
Iteration 4 ETA: [00:02:46]
Iteration 5 ETA: [00:02:46]
Iteration 6 ETA: [00:02:40]
Iteration 7 ETA: [00:02:38]
Iteration 8 ETA: [00:02:35]
Iteration 9 ETA: [00:02:33]
Iteration 10 ETA: [00:02:32]
Iteration 11 ETA: [00:02:32]
Iteration 12 ETA: [00:02:29]
Iteration 13 ETA: [00:02:25]
Iteration 14 ETA: [00:02:29]
Iteration 15 ETA: [00:02:23]
Iteration 16 ETA: [00:02:22]
Iteration 17 ETA: [00:02:23]
Iteration 18 ETA: [00:02:19]
Iteration 19 ETA: [00:02:19]
Iteration 20 ETA: [00:02:19]
Iteration 21 ETA: [00:02:15]
Iteration 22 ETA: [00:02:15]
Iteration 23 ETA: [00:02:12]
Iteration 24 ETA: [00:02:08]
Iteration 25 ETA: [00:02:08]
Iteration 26 ETA: [00:02:08]
Iteration 27 ETA: [00:02:07]
Iteration 28 ETA: [00:02:02]
Iteration 29 ETA: [00:02:02]
Iteration 30 ETA: [

In [21]:
order_0 = torch.cat(order_0, dim=0)
orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

# Regression

In [22]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

In [1]:
# basename = 'molecule_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
#         L, J, sigma, (M, N, O), integral_powers)

# cache_dir = get_cache_dir("ml_projet/")

# filename = os.path.join(cache_dir, 'order_0_' + basename)
# np.save(filename, order_0)

# filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
# np.save(filename, orders_1_and_2)

In [23]:
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)

In [24]:
target = energies_df['energy'].to_list()

In [25]:
n_folds = 3

P = np.random.permutation(n_molecules).reshape((n_folds, -1))

cross_val_folds = []

for i_fold in range(n_folds):
    fold = (np.concatenate(P[np.arange(n_folds) != i_fold], axis=0),
            P[i_fold])
    cross_val_folds.append(fold)

In [38]:
# from sklearn import preprocessing, model_selection, pipeline, neural_network
# from sklearn.metrics import mean_absolute_error, mean_squared_error


In [36]:
# alphas = 10.0 ** (-np.arange(1, 10))  # on garde alphas juste pour la structure

# # Test avec MLPRegressor
# for i, alpha in enumerate(alphas):
#     scaler = preprocessing.StandardScaler()
    
#     mlp = neural_network.MLPRegressor(
#         hidden_layer_sizes=(100,),
#         activation='relu',
#         alpha=alpha,  # régularisation L2
#         max_iter=1000,
#         random_state=0
#     )

#     regressor = pipeline.make_pipeline(scaler, mlp)

#     target_prediction = model_selection.cross_val_predict(
#         regressor,
#         X=scattering_coef, y=target,
#         cv=cross_val_folds
#     )

#     MAE = mean_absolute_error(target, target_prediction)
#     RMSE = np.sqrt(mean_squared_error(target, target_prediction))

#     print(f'MLP regression, alpha: {alpha}, MAE: {MAE:.4f}, RMSE: {RMSE:.4f}')

In [37]:
# import numpy as np
# from sklearn import preprocessing, model_selection, pipeline
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# # Paramètres
# n_folds = 3
# n_estimators = 50  # Nombre d'arbres dans la forêt

# # Création des folds de validation croisée
# P = np.random.permutation(n_molecules).reshape((n_folds, -1))
# cross_val_folds = [
#     (np.concatenate(P[np.arange(n_folds) != i], axis=0), P[i])
#     for i in range(n_folds)
# ]

# # Initialisation du modèle
# scaler = preprocessing.StandardScaler()  # Pas obligatoire, mais inoffensif ici
# rf = RandomForestRegressor(n_estimators=n_estimators, random_state=0)

# # Pipeline (tu peux aussi retirer le scaler si tu veux)
# regressor = pipeline.make_pipeline(scaler, rf)

# # Prédiction avec validation croisée
# target_prediction = model_selection.cross_val_predict(
#     regressor,
#     X=scattering_coef, y=target,
#     cv=cross_val_folds
# )

# # Évaluation
# MAE = mean_absolute_error(target, target_prediction)
# RMSE = np.sqrt(mean_squared_error(target, target_prediction))

# print(f'Random Forest regression, MAE: {MAE:.4f}, RMSE: {RMSE:.4f}')


In [29]:
alphas = 10.0 ** (-np.arange(1, 10))
for i, alpha in enumerate(alphas):
    scaler = preprocessing.StandardScaler()
    ridge = linear_model.Ridge(alpha=alpha)

    regressor = pipeline.make_pipeline(scaler, ridge)

    target_prediction = model_selection.cross_val_predict(regressor,
            X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    print('Ridge regression, alpha: {}, MAE: {}, RMSE: {}'.format(
        alpha, MAE, RMSE))

Ridge regression, alpha: 0.1, MAE: 9.66172895229075, RMSE: 11.720544834581451
Ridge regression, alpha: 0.01, MAE: 9.68187271689484, RMSE: 11.749491712067872


/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.71834e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.36847e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.96315e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge regression, alpha: 0.001, MAE: 9.709746322747444, RMSE: 11.786689229310761
Ridge regression, alpha: 0.0001, MAE: 9.726457380547803, RMSE: 11.811213122764853
Ridge regression, alpha: 1e-05, MAE: 9.732403065270674, RMSE: 11.820604179851724
Ridge regression, alpha: 1e-06, MAE: 9.737637633045809, RMSE: 11.830904890960003
Ridge regression, alpha: 1e-07, MAE: 9.735915321665608, RMSE: 11.831444838889238
Ridge regression, alpha: 1e-08, MAE: 9.73812517627796, RMSE: 11.837737414933907
Ridge regression, alpha: 1e-09, MAE: 9.747173451948845, RMSE: 11.849300619055338


In [26]:
import numpy as np
import pandas as pd
import torch
from sklearn import preprocessing, linear_model, pipeline, model_selection

# Étape 1 : Définir les alphas
alphas = 10.0 ** (-np.arange(1, 10))
best_mae = float('inf')
best_alpha = None
best_regressor = None

# Tes données d'entraînement
X = scattering_coef  # numpy array
y = target            # numpy array
cross_val_folds = 5   # ou une autre valeur selon ton besoin

# Recherche du meilleur alpha
for alpha in alphas:
    scaler = preprocessing.StandardScaler()
    ridge = linear_model.Ridge(alpha=alpha)
    regressor = pipeline.make_pipeline(scaler, ridge)

    target_prediction = model_selection.cross_val_predict(regressor, X=X, y=y, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - y))
    RMSE = np.sqrt(np.mean((target_prediction - y) ** 2))

    print(f'Ridge regression, alpha: {alpha}, MAE: {MAE}, RMSE: {RMSE}')

    if MAE < best_mae:
        best_mae = MAE
        best_alpha = alpha
        best_regressor = regressor

# Étape 2 : Réentraîner le meilleur pipeline sur toutes les données
print(f'Best alpha selected: {best_alpha}')
best_regressor.fit(X, y)

Ridge regression, alpha: 0.1, MAE: 9.660161972779612, RMSE: 11.712241073785105


/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.89664e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.05602e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.44759e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge regression, alpha: 0.01, MAE: 9.669674170337464, RMSE: 11.731424644521882
Ridge regression, alpha: 0.001, MAE: 9.696526951904591, RMSE: 11.761996635861857
Ridge regression, alpha: 0.0001, MAE: 9.723082230372942, RMSE: 11.798352808098604
Ridge regression, alpha: 1e-05, MAE: 9.734905479474499, RMSE: 11.819747260704862
Ridge regression, alpha: 1e-06, MAE: 9.747007240729106, RMSE: 11.848899210384998
Ridge regression, alpha: 1e-07, MAE: 9.753425244640692, RMSE: 11.866700369202182
Ridge regression, alpha: 1e-08, MAE: 9.757285770441904, RMSE: 11.89218343573741
Ridge regression, alpha: 1e-09, MAE: 9.762874554915413, RMSE: 11.90594535261694
Best alpha selected: 0.1


/home/sara/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=5.90879e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.1))])

# test

In [28]:
pos, full_charges = load_all_xyz("./atoms/test/", max_atoms=23)
n_molecules = pos.shape[0]

In [29]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

In [30]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

In [31]:
M, N, O = 32, 32, 32

grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [32]:
J = 1
L = 2
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

In [33]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)

HarmonicScattering3D()

In [34]:
batch_size = 64
n_batches = int(np.ceil(n_molecules / batch_size))
print(n_batches)

26


In [35]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 1647 molecules from the QM7 database on GPU
sigma: 2.0, L: 2, J: 1, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:00:39]
Iteration 3 ETA: [00:00:37]
Iteration 4 ETA: [00:00:36]
Iteration 5 ETA: [00:00:34]
Iteration 6 ETA: [00:00:32]
Iteration 7 ETA: [00:00:31]
Iteration 8 ETA: [00:00:29]
Iteration 9 ETA: [00:00:28]
Iteration 10 ETA: [00:00:26]
Iteration 11 ETA: [00:00:25]
Iteration 12 ETA: [00:00:23]
Iteration 13 ETA: [00:00:21]
Iteration 14 ETA: [00:00:20]
Iteration 15 ETA: [00:00:17]
Iteration 16 ETA: [00:00:16]
Iteration 17 ETA: [00:00:15]
Iteration 18 ETA: [00:00:13]
Iteration 19 ETA: [00:00:11]
Iteration 20 ETA: [00:00:09]
Iteration 21 ETA: [00:00:08]
Iteration 22 ETA: [00:00:06]
Iteration 23 ETA: [00:00:04]
Iteration 24 ETA: [00:00:03]
Iteration 25 ETA: [00:00:01]
Iteration 26 ETA: [00:00:00]


In [36]:
order_0 = torch.cat(order_0, dim=0)
orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

In [37]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

In [38]:
scattering_coef_test = np.concatenate([order_0, orders_1_and_2], axis=1)

In [43]:
predictions = best_regressor.predict(scattering_coef_test)

results = pd.DataFrame({
    'id': np.arange(len(predictions)),
    'energy': predictions
})
results = results.sort_values(by='energy').reset_index(drop=True)
results['id'] = results['id'] + 6592
results.to_csv('predictions.csv', index=False)

In [44]:
results

,id,energy
0,7424,-82.951691
1,7520,-82.856049
2,6682,-82.324692
3,6761,-81.625381
4,7503,-81.058838
...,...,...
1642,6759,-74.520844
1643,7307,-74.437408
1644,7824,-74.249481
1645,8005,-74.048737
